# Expand the incident wave using sphercial wave functions

In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode#
import numba
import math
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

In [147]:
h = 0.2 # Size of the mesh
d = [1,0,0] # wave's travel direction
k = 0.2 # wavenumber

In [61]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [4]:
def coeff_sph_expansion(p,q):
    """Analytical formula of the coefficients of the spherical expansions of plane waves"""
    coeff = 4*np.pi*((1j)**q)*normalized_spherical_harmonics(-p,q,d)/(-1)**(abs(p))
    return coeff

In [79]:
dom_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_sphere = dom_sphere.vertices
space_sphere = bempp.api.function_space(dom_sphere, 'P', 1)

### Expand $u$ into known basis with known coefficient on the sphere

#### $u = \sum_{i}\alpha_{i}\phi_{i}$ and we set $(\alpha_{i}) = \alpha$

In [80]:
@bempp.api.complex_callable
def inc_wave_fun(x,n,domain_index, result):
    result[0] = np.exp(1j*k*x[0])
    
inc_wave_grid_fun_sphere = bempp.api.GridFunction(space_sphere, fun = inc_wave_fun)
inc_wave_grid_coeff_sphere = inc_wave_grid_fun_sphere.coefficients

In [81]:
mass_mat = bempp.api.operators.boundary.sparse.identity(space_sphere,space_sphere,space_sphere).weak_form().A

In [82]:
@bempp.api.complex_callable
def test_fun(x,n,domain_index, result):
    with objmode():
        result[0] = 1*normalized_spherical_harmonics(1,2,x) + 2*normalized_spherical_harmonics(3,4,x)
    
test_grid_fun = bempp.api.GridFunction(space_sphere, fun = test_fun)
test_fun_coeff = test_grid_fun.coefficients

In [35]:
@bempp.api.complex_callable
def basis_fun12(x,n,domain_index,result):
    with objmode():
        result[0] = normalized_spherical_harmonics(1,2,x)

basis_grid12 = bempp.api.GridFunction(space_sphere,fun=basis_fun12)
basis_coeff12 = basis_grid12.coefficients

@bempp.api.complex_callable
def basis_fun34(x,n,domain_index,result):
    with objmode():
        result[0] = normalized_spherical_harmonics(3,4,x)

basis_grid34 = bempp.api.GridFunction(space_sphere,fun=basis_fun34)
basis_coeff34 = basis_grid34.coefficients

@bempp.api.complex_callable
def basis_fun56(x,n,domain_index,result):
    with objmode():
        result[0] = normalized_spherical_harmonics(5,6,x)

basis_grid56 = bempp.api.GridFunction(space_sphere,fun=basis_fun56)
basis_coeff56 = basis_grid56.coefficients

In [67]:
(test_fun_coeff @ mass_mat @ np.conj(basis_coeff12))*normalized_spherical_harmonics(1,2,vert_sphere[:,55]) + (test_fun_coeff @ mass_mat @ np.conj(basis_coeff34))*normalized_spherical_harmonics(3,4,vert_sphere[:,55])

(-0.765753524300927+2.0308074889490185e-08j)

In [70]:
1*normalized_spherical_harmonics(1,2,vert_sphere[:,55]) + 2*normalized_spherical_harmonics(3,4,vert_sphere[:,55])

(-0.7770497895765304+2.3817851489619574e-16j)

In [83]:
deg = 10
coeff_sph_wf = []
for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def basis_fun(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)

        basis_grid = bempp.api.GridFunction(space_sphere,fun=basis_fun)
        basis_coeff = basis_grid.coefficients
        coeff_sph_wf.append(basis_coeff)

In [148]:
def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)

def spherical_hankel_function(q,x):
    """Spherical hankel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.hankel1(q+0.5, k*r)

def Legendre_poly(q, x):  
    if(q == 0): 
        return 1 # P0 = 1 
    elif(q == 1): 
        return x # P1 = x 
    else: 
        return ((2*q - 1) * x * Legendre_poly(q-1, x) - (q-1)*Legendre_poly(q - 2, x))/q 

def far_field(x):
    field = 0
    radius = 4
    cosine_angle = x[0]/np.linalg.norm(x)
    for i in range(20):
        a = (1j/k) * (2*i+1) * spherical_bessel_function(i,[radius,0,0]) * Legendre_poly(i,cosine_angle) / spherical_hankel_function(i,[radius,0,0])
        field += a
    return field

In [149]:
@bempp.api.complex_callable
def far_fun(x,n,domain_index, result):
    with objmode():
        result[0] = far_field(x)
    
far_fun_sphere = bempp.api.GridFunction(space_sphere, fun = far_fun)
far_grid_coeff_sphere = far_fun_sphere.coefficients

In [150]:
appro_far_list = []
for i in range(len(coeff_sph_wf)):
    appro_far_list.append(far_grid_coeff_sphere @ mass_mat @ np.conj(coeff_sph_wf[i]))

In [84]:
appro_list = []
for i in range(len(coeff_sph_wf)):
    appro_list.append(inc_wave_grid_coeff_sphere @ mass_mat @ np.conj(coeff_sph_wf[i]))

In [85]:
def inc_field(x):
    fun = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            fun += appro_list[i]* normalized_spherical_harmonics(p, q, x)
            i +=1
    return fun

In [151]:
def far_field_appro(x):
    fun = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            fun += appro_far_list[i]* normalized_spherical_harmonics(p, q, x)
            i +=1
    return fun

In [152]:
far_field_appro([1,0,0])

(-4.493267490841373+2.7936025635631014j)

In [153]:
far_field([1,0,0])

(-4.517459973298983+2.8080790803114657j)

In [135]:
@bempp.api.complex_callable
def bessel_fun(x,n,domain_index, result):
    with objmode():
        result[0] = spherical_hankel_function(1,x)
    
bessel_fun_sphere = bempp.api.GridFunction(space_sphere, fun = bessel_fun)
bessel_grid_coeff_sphere = bessel_fun_sphere.coefficients

In [136]:
appro_bessel_list = []
for i in range(len(coeff_sph_wf)):
    appro_bessel_list.append(bessel_grid_coeff_sphere @ mass_mat @ np.conj(coeff_sph_wf[i]))

In [137]:
def bessel_appro(x):
    fun = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            fun += appro_bessel_list[i]* normalized_spherical_harmonics(p, q, x)
            i +=1
    return fun

In [138]:
bessel_appro([4,0,0])

(-0.09199434488200976+0.18166934727027587j)

In [139]:
spherical_bessel_function(1,[4,0,0])

-0.018121739963850764